# Comparison Between TreeValue and Tianshou Batch

In this section, we will take a look at the feature and performance of the [Tianshou Batch](https://github.com/thu-ml/tianshou) library, which is developed by Tsinghua Machine Learning Group.

Before starting the comparison, let us define some thing.

In [1]:
import torch 

_TREE_DATA_1 = {'a': 1, 'b': 2, 'x': {'c': 3, 'd': 4}}
_TREE_DATA_2 = {
    'a': torch.randn(2, 3), 
    'x': {
        'c': torch.randn(3, 4)
    },
}
_TREE_DATA_3 = {
    'obs': torch.randn(4, 84, 84),
    'action': torch.randint(0, 6, size=(1,)),
    'reward': torch.rand(1),
}

/opt/hostedtoolcache/Python/3.8.16/x64/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Read and Write Operation

Reading and writing are the two most common operations in the tree data structure based on the data model (TreeValue and Tianshou Batch both belong to this type), so this section will compare the reading and writing performance of these two libraries.

### TreeValue's Get and Set

In [2]:
from treevalue import FastTreeValue

t = FastTreeValue(_TREE_DATA_2)

In [3]:
t

<FastTreeValue 0x7f4e20799220>
├── 'a' --> tensor([[ 1.5305,  1.0268, -0.2935],
│                   [ 1.3295, -0.6799, -0.0527]])
└── 'x' --> <FastTreeValue 0x7f4e20799460>
    └── 'c' --> tensor([[ 0.1379, -0.9410,  0.8959,  1.2480],
                        [-0.6514,  1.7412, -1.4428,  0.8409],
                        [ 0.4519, -1.1349, -0.6025, -0.4864]])

In [4]:
t.a

tensor([[ 1.5305,  1.0268, -0.2935],
        [ 1.3295, -0.6799, -0.0527]])

In [5]:
%timeit t.a

60.2 ns ± 0.0337 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [6]:
new_value = torch.randn(2, 3)
t.a = new_value

t

<FastTreeValue 0x7f4e20799220>
├── 'a' --> tensor([[ 1.1694, -0.4244,  0.6637],
│                   [-0.5413,  0.1728, -1.2540]])
└── 'x' --> <FastTreeValue 0x7f4e20799460>
    └── 'c' --> tensor([[ 0.1379, -0.9410,  0.8959,  1.2480],
                        [-0.6514,  1.7412, -1.4428,  0.8409],
                        [ 0.4519, -1.1349, -0.6025, -0.4864]])

In [7]:
%timeit t.a = new_value

60 ns ± 0.145 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


### Tianshou Batch's Get and Set

In [8]:
from tianshou.data import Batch

b = Batch(**_TREE_DATA_2)

In [9]:
b

Batch(
    a: tensor([[ 1.5305,  1.0268, -0.2935],
               [ 1.3295, -0.6799, -0.0527]]),
    x: Batch(
           c: tensor([[ 0.1379, -0.9410,  0.8959,  1.2480],
                      [-0.6514,  1.7412, -1.4428,  0.8409],
                      [ 0.4519, -1.1349, -0.6025, -0.4864]]),
       ),
)

In [10]:
b.a

tensor([[ 1.5305,  1.0268, -0.2935],
        [ 1.3295, -0.6799, -0.0527]])

In [11]:
%timeit b.a

59 ns ± 0.0261 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [12]:
new_value = torch.randn(2, 3)
b.a = new_value

b

Batch(
    a: tensor([[-0.5514, -0.3660, -0.4694],
               [-1.1623,  0.5158, -0.9948]]),
    x: Batch(
           c: tensor([[ 0.1379, -0.9410,  0.8959,  1.2480],
                      [-0.6514,  1.7412, -1.4428,  0.8409],
                      [ 0.4519, -1.1349, -0.6025, -0.4864]]),
       ),
)

In [13]:
%timeit b.a = new_value

488 ns ± 0.175 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


## Initialization

### TreeValue's Initialization

In [14]:
%timeit FastTreeValue(_TREE_DATA_1)

845 ns ± 0.501 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


### Tianshou Batch's Initialization

In [15]:
%timeit Batch(**_TREE_DATA_1)

11.3 µs ± 26.2 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


## Deep Copy Operation

In [16]:
import copy

### Deep Copy of TreeValue

In [17]:
t3 = FastTreeValue(_TREE_DATA_3)
%timeit copy.deepcopy(t3)

138 µs ± 631 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Deep Copy of Tianshou Batch

In [18]:
b3 = Batch(**_TREE_DATA_3)
%timeit copy.deepcopy(b3)

142 µs ± 987 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


## Stack, Concat and Split Operation

### Performance of TreeValue

In [19]:
trees = [FastTreeValue(_TREE_DATA_2) for _ in range(8)]

In [20]:
t_stack = FastTreeValue.func(subside=True)(torch.stack)

t_stack(trees)

<FastTreeValue 0x7f4e2071f640>
├── 'a' --> tensor([[[ 1.5305,  1.0268, -0.2935],
│                    [ 1.3295, -0.6799, -0.0527]],
│           
│                   [[ 1.5305,  1.0268, -0.2935],
│                    [ 1.3295, -0.6799, -0.0527]],
│           
│                   [[ 1.5305,  1.0268, -0.2935],
│                    [ 1.3295, -0.6799, -0.0527]],
│           
│                   [[ 1.5305,  1.0268, -0.2935],
│                    [ 1.3295, -0.6799, -0.0527]],
│           
│                   [[ 1.5305,  1.0268, -0.2935],
│                    [ 1.3295, -0.6799, -0.0527]],
│           
│                   [[ 1.5305,  1.0268, -0.2935],
│                    [ 1.3295, -0.6799, -0.0527]],
│           
│                   [[ 1.5305,  1.0268, -0.2935],
│                    [ 1.3295, -0.6799, -0.0527]],
│           
│                   [[ 1.5305,  1.0268, -0.2935],
│                    [ 1.3295, -0.6799, -0.0527]]])
└── 'x' --> <FastTreeValue 0x7f4e2071f1f0>
    └── 'c' --> tensor([[[

In [21]:
%timeit t_stack(trees)

32.4 µs ± 38.4 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [22]:
t_cat = FastTreeValue.func(subside=True)(torch.cat)

t_cat(trees)

<FastTreeValue 0x7f4df4bb1190>
├── 'a' --> tensor([[ 1.5305,  1.0268, -0.2935],
│                   [ 1.3295, -0.6799, -0.0527],
│                   [ 1.5305,  1.0268, -0.2935],
│                   [ 1.3295, -0.6799, -0.0527],
│                   [ 1.5305,  1.0268, -0.2935],
│                   [ 1.3295, -0.6799, -0.0527],
│                   [ 1.5305,  1.0268, -0.2935],
│                   [ 1.3295, -0.6799, -0.0527],
│                   [ 1.5305,  1.0268, -0.2935],
│                   [ 1.3295, -0.6799, -0.0527],
│                   [ 1.5305,  1.0268, -0.2935],
│                   [ 1.3295, -0.6799, -0.0527],
│                   [ 1.5305,  1.0268, -0.2935],
│                   [ 1.3295, -0.6799, -0.0527],
│                   [ 1.5305,  1.0268, -0.2935],
│                   [ 1.3295, -0.6799, -0.0527]])
└── 'x' --> <FastTreeValue 0x7f4df4bb1bb0>
    └── 'c' --> tensor([[ 0.1379, -0.9410,  0.8959,  1.2480],
                        [-0.6514,  1.7412, -1.4428,  0.8409],
                 

In [23]:
%timeit t_cat(trees)

30.4 µs ± 50.8 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [24]:
t_split = FastTreeValue.func(rise=True)(torch.split)
tree = FastTreeValue({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1),
})

%timeit t_split(tree, 1)

57.4 µs ± 96 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Performance of Tianshou Batch

In [25]:
batches = [Batch(**_TREE_DATA_2) for _ in range(8)]

Batch.stack(batches)

Batch(
    a: tensor([[[ 1.5305,  1.0268, -0.2935],
                [ 1.3295, -0.6799, -0.0527]],
       
               [[ 1.5305,  1.0268, -0.2935],
                [ 1.3295, -0.6799, -0.0527]],
       
               [[ 1.5305,  1.0268, -0.2935],
                [ 1.3295, -0.6799, -0.0527]],
       
               [[ 1.5305,  1.0268, -0.2935],
                [ 1.3295, -0.6799, -0.0527]],
       
               [[ 1.5305,  1.0268, -0.2935],
                [ 1.3295, -0.6799, -0.0527]],
       
               [[ 1.5305,  1.0268, -0.2935],
                [ 1.3295, -0.6799, -0.0527]],
       
               [[ 1.5305,  1.0268, -0.2935],
                [ 1.3295, -0.6799, -0.0527]],
       
               [[ 1.5305,  1.0268, -0.2935],
                [ 1.3295, -0.6799, -0.0527]]]),
    x: Batch(
           c: tensor([[[ 0.1379, -0.9410,  0.8959,  1.2480],
                       [-0.6514,  1.7412, -1.4428,  0.8409],
                       [ 0.4519, -1.1349, -0.6025, -0.4864]],
         

In [26]:
%timeit Batch.stack(batches)

80.1 µs ± 257 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [27]:
Batch.cat(batches)

Batch(
    a: tensor([[ 1.5305,  1.0268, -0.2935],
               [ 1.3295, -0.6799, -0.0527],
               [ 1.5305,  1.0268, -0.2935],
               [ 1.3295, -0.6799, -0.0527],
               [ 1.5305,  1.0268, -0.2935],
               [ 1.3295, -0.6799, -0.0527],
               [ 1.5305,  1.0268, -0.2935],
               [ 1.3295, -0.6799, -0.0527],
               [ 1.5305,  1.0268, -0.2935],
               [ 1.3295, -0.6799, -0.0527],
               [ 1.5305,  1.0268, -0.2935],
               [ 1.3295, -0.6799, -0.0527],
               [ 1.5305,  1.0268, -0.2935],
               [ 1.3295, -0.6799, -0.0527],
               [ 1.5305,  1.0268, -0.2935],
               [ 1.3295, -0.6799, -0.0527]]),
    x: Batch(
           c: tensor([[ 0.1379, -0.9410,  0.8959,  1.2480],
                      [-0.6514,  1.7412, -1.4428,  0.8409],
                      [ 0.4519, -1.1349, -0.6025, -0.4864],
                      [ 0.1379, -0.9410,  0.8959,  1.2480],
                      [-0.6514,  

In [28]:
%timeit Batch.cat(batches)

143 µs ± 1.85 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [29]:
batch = Batch({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1)}
)

%timeit list(Batch.split(batch, 1, shuffle=False, merge_last=True))

308 µs ± 6.9 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
